# Datasets 2020
Para cada año tenemos 3 tablas a inspeccionar, limpiar y unir:
- `title_basics`: pública y descargada de IMDb
- `title_ratings`: pública y descargada de IMDb
- `movies`: propia, escrapeada de la web de IMDb para todas las películas del año correspondiente

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
anno = 2020

In [3]:
# Tabla descargada de IMDb

title_basics = pd.read_csv('../data/imdb/title_basics.tsv', sep='\t', na_values='\\N')
print(title_basics.shape)
title_basics.head()

C:\Users\casiopa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(8084314, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short"


In [4]:
# Tabla descargada de IMDb

title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286


In [5]:
# Películas de escrapeadas de la web IMDb

movies = pd.read_csv('../data/web_imdb/movies_df_'+ str(anno) + '.csv', sep=';', na_values=["[]", "['']", "['', '']", "['', '', '']"])
print(movies.shape)
movies.head()

(14632, 20)


,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
0,tt0062336,El Tango del Viudo y Su Espejo Deformante,NaN,2020.0,NaN,NaN,"['Raoul Ruiz', 'Valeria Sarmiento']","['Raoul Ruiz', 'Omar Saavedra Santis']","['Rubén Sotoconil', 'Claudia Paz', 'Luis Alarc...",['Drama'],['Chile'],NaN,6.3,NaN,NaN,Awards\n1 nomination,NaN,NaN,NaN,NaN
1,tt0065392,Amintiri bucurestene,NaN,2020.0,NaN,NaN,"['Yousif Mohamed Aidaby', 'Maria Alexandru', '...",NaN,NaN,['Documentary'],['Romania'],NaN,7.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0093119,Grizzly II: El concierto,Grizzly II: The Predator,2020.0,Not Rated,1h 14min,['André Szöts'],"['Joan McCall', 'David Sheldon']","['George Clooney', 'Laura Dern', 'Charlie Sheen']","['Horror', 'Music', 'Thriller']","['United States', 'Hungary']",['GBGB International'],3.3,7.0,"4,092",NaN,"$7,500,000",NaN,NaN,NaN
3,tt0293513,Pit Bull,NaN,2020.0,NaN,NaN,['Aubrey K. Rattan'],['Aubrey K. Rattan'],"['Cynthea Amanson', 'Patti Been', 'Jerry Boyd']",NaN,['United States'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0368133,The Promise of Perfume,NaN,2020.0,NaN,NaN,['Pavlina Proevska'],['Pavlina Proevska'],"['Elizabeth Bluhm', 'Lara', 'Esthir Marks']",['Documentary'],['United States'],['Pavlina Ltd.'],6.6,NaN,NaN,NaN,"$200,000",NaN,NaN,NaN


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14632 entries, 0 to 14631
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   imdb_id            14632 non-null  object 
 1   title              14623 non-null  object 
 2   original_title     538 non-null    object 
 3   year               14536 non-null  float64
 4   certificate        1659 non-null   object 
 5   duration           1610 non-null   object 
 6   directors          14151 non-null  object 
 7   writers            13007 non-null  object 
 8   stars              9480 non-null   object 
 9   genres             14166 non-null  object 
 10  countries          14165 non-null  object 
 11  companies          10859 non-null  object 
 12  imdb_rating        6744 non-null   float64
 13  metascore          581 non-null    float64
 14  popularity         314 non-null    object 
 15  awards             3131 non-null   object 
 16  budget             449

---
## Número de películas

En primer lugar verificar si hay una discrepancia entre el número de películas obtenidas de title_basics.tsv de IMDb
y el número de películas resultantes del scrapping.

In [7]:
# Cargar las películas de la base de datos de IMDb en una lista

imdb_ids = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]
imdb_ids = list(imdb_ids['tconst'])
len(imdb_ids)

14632

In [8]:
scrap_ids = list(movies['imdb_id'])
len(scrap_ids)

14632

---
## 1. Bd de IMDb: `title_basics`

### Variable `runtimeMinutes`  

In [9]:
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8084314 entries, 0 to 8084313
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   isAdult         float64
 5   startYear       float64
 6   endYear         float64
 7   runtimeMinutes  object 
 8   genres          object 
dtypes: float64(3), object(6)
memory usage: 555.1+ MB


#### Convertir `runtimeMinutes` en float
La siguiente función convierte los valores en float si se puede de manera natural, y si no se puede los imprime y convierte a nulo.

In [10]:
def to_float(n):
    try:
        n = float(n)
        return n
    except:
        print(n)
        return np.nan

In [11]:
title_basics.runtimeMinutes = title_basics.runtimeMinutes.apply(to_float)
title_basics.runtimeMinutes

Reality-TV
Documentary
Talk-Show
Game-Show
Reality-TV
Animation,Comedy,Family
Reality-TV


0           1.0
1           5.0
2           4.0
3          12.0
4           1.0
           ... 
8084309     NaN
8084310     NaN
8084311     NaN
8084312    27.0
8084313    10.0
Name: runtimeMinutes, Length: 8084314, dtype: float64

#### Eliminar de `title_basics` los registros con outliers en `runtimeMinutes`

In [12]:
# Buscar outliers mayores de 1.000 minutos y menores 15
# El resltado es un listado de películas fake, cortos o compendios de pelis.
# Eliminar todos estos registros del scrapping

title_basics = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]

title_basics = title_basics[~((title_basics.runtimeMinutes<15) | (title_basics.runtimeMinutes>500))]
title_basics.shape

(14623, 9)

In [13]:
# Una vez eliminados los outliers estos son los estadísticos:

title_basics.runtimeMinutes.describe().round(2)

count    10407.00
mean        89.18
std         26.71
min         15.00
25%         74.00
50%         89.00
75%        101.00
max        480.00
Name: runtimeMinutes, dtype: float64

---
## 2. Bd de IMDb: `title_ratings`

In [14]:
title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286



---
## 3. Bd propia escrapeada del portal IMDb: `movies`

## `year`
La base de datos de IMDb descargada tiene algunas discrepancias en cuanto al año de las películas, no coindice el año en la base de datos descargada de IMDb con el año del portal web, en el caso del año 2019 ocurre en 12 registros. Existe algún caso en el que tampoco coincide el año de la película en el portal Metacritic. Hay varias formas de afrontar estas discrepancias, en este caso, y por simplificación, utilizaré el año de la base de datos de IMDb.
Hay también algunos registros en la base de datos con valores nulos para el año y demás columnas, y serán eliminados (47 registors en total).

In [15]:
# Películas escrapeadas con año diferente
len(movies[(movies['year']!=anno) & movies['year'].notnull()])

50

In [16]:
movies[(movies['year']!=anno) & movies['year'].notnull()]

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
129,tt10125706,Beer Pioneers,NaN,2022.0,NaN,NaN,['Jay Sheveck'],"['Jim Brickman', 'Garrett Oliver', 'Michael Ja...",NaN,['Documentary'],['United States'],"['Add/Edit', 'Beer Guppy Multimedia', 'Earthbo...",NaN,NaN,NaN,NaN,"$750,000",NaN,NaN,NaN
589,tt10447368,Pompano Boy,NaN,2021.0,NaN,NaN,['Andrew Paul Davis'],"['Andrew Paul Davis', 'Timothy Mark Davis']","['Garry Pastore', 'Shein Mompremier', 'Todd Br...",['Drama'],['United States'],['One County Film Company'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950,tt10674614,Ashley Jones Is Perfectly Normal,NaN,2021.0,NaN,NaN,['Thomas Bangert'],"['Thomas Bangert', 'Kev Stock']","['Hana Yuka Sano', 'Loa Allebach', 'Chris Yesc...","['Comedy', 'Drama']",['United States'],"['CronicLove Productions', 'Platform12', 'Trip...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
982,tt10687866,Royaltee Studios Presents: Thy Kingdom Com,NaN,2021.0,NaN,NaN,['Willie Jae'],['Willie Jae'],"['Jade Mead', 'Jade Mead', 'Willie Jae']",['Documentary'],['United States'],['Royaltee Management'],NaN,NaN,NaN,NaN,"$25,000",NaN,NaN,NaN
1046,tt10731778,Malignant,NaN,2021.0,R,1h 51min,['James Wan'],"['Akela Cooper', 'James Wan', 'Ingrid Bisu']","['Annabelle Wallis', 'Maddie Hasson', 'George ...","['Horror', 'Thriller']",['United States'],"['Atomic Monster', 'Boom Entertainment', 'Boom...",NaN,NaN,"1,560",NaN,NaN,NaN,NaN,NaN
1198,tt1083463,Marilyn Monroe: Murder on Fifth Helena Drive,NaN,2021.0,NaN,NaN,['Keya Morgan'],['Keya Morgan'],"['Keya Morgan', 'Marilyn Monroe', 'Richard Bel...",['Documentary'],['United States'],['Keya Morgan Productions'],NaN,NaN,NaN,NaN,"$2,000,000",NaN,NaN,NaN
1241,tt10873536,Ninna Sanihake,NaN,2021.0,NaN,NaN,['Suraj Gowda'],"['Suraj Gowda', 'Praveen Kumar G']","['Suraj Gowda', 'Dhanya Ramkumar', 'Aruna Balr...","['Drama', 'Romance']",['India'],['White and Grey Pictures'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1352,tt10937004,The Last Planet,The Way of the Wind,2021.0,NaN,NaN,['Terrence Malick'],['Terrence Malick'],"['Aidan Turner', 'Joseph Fiennes', 'Douglas Bo...","['Drama', 'History']",['United States'],['Babelsberg Film'],NaN,NaN,"4,484",NaN,NaN,NaN,NaN,NaN
1361,tt10940848,Un mundo complicado,Jagame Thandhiram,2021.0,16,2h 38min,['Karthik Subbaraj'],"['Nick Bain', 'Karthik Subbaraj']","['Dhanush', 'James Cosmo', 'Aishwarya Lekshmi']","['Action', 'Crime', 'Drama']","['United Kingdom', 'India']","['Reliance Entertainment', 'Y Not Studios']",6.0,NaN,"1,060",NaN,"£7,995,000",NaN,NaN,NaN
1576,tt11061084,The Mass Shooting Monologues,NaN,2021.0,NaN,NaN,['Brian Ronalds'],"['Michelle Palermo', 'Brian Ronalds']","['Vai Patri', 'Leif Ronalds', 'Tom Malloy']",['Drama'],['United States'],"['Dark Falls PR', 'Forty Four Eleven Productio...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Eliminar del df escrapeado las películas de años posteriores a 2020
(son pelis futuras que se han colado)

In [17]:
movies = movies[~(movies['year']>2020)]
len(movies)

14590

#### Eliminar del df escrapeado las películas sin año
(son pelis sin información o url en el portal)

In [18]:
# Películas escrapeadas con año null, no tienen casi información, serán eliminadas
movies = movies[movies['year'].notnull()]
len(movies)

14494

## `imdb_rating`
Esta variable será sustituida por la de la base de datos `title_rating` descargada de IMDb

In [19]:
movies[['imdb_rating']].describe().round(2)

,imdb_rating
count,6734.00
mean,6.21
std,1.63
min,1.00
25%,5.20
50%,6.30
75%,7.40
max,10.00


## `metascore`

In [20]:
movies[['imdb_id', 'title', 'original_title', 'imdb_rating', 'metascore', 'year']][movies.metascore>=50].sort_values('metascore', ascending=False)

,imdb_id,title,original_title,imdb_rating,metascore,year
13633,tt8633462,"Quo vadis, Aida?",NaN,7.9,97.0,2020.0
3943,tt11874226,American Utopia,NaN,8.2,93.0,2020.0
14511,tt9770150,Nomadland,NaN,7.4,93.0,2020.0
13125,tt7772582,"Nunca, casi nunca, a veces, siempre",Never Rarely Sometimes Always,7.4,91.0,2020.0
2293,tt11416746,Time,NaN,6.8,91.0,2020.0
...,...,...,...,...,...,...
11889,tt2222052,Sueños de una escritora en Nueva York,My Salinger Year,6.3,50.0,2020.0
13606,tt8580274,Festival de la Canción de Eurovisión: La histo...,Eurovision Song Contest: The Story of Fire Saga,6.5,50.0,2020.0
231,tt10213380,Malmkrog,NaN,6.2,50.0,2020.0
13417,tt8310476,The Violent Heart,NaN,5.4,50.0,2020.0


In [21]:
np.sort(movies.metascore.dropna().unique())

array([ 7., 11., 13., 14., 15., 20., 22., 24., 25., 26., 27., 29., 30.,
       31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43.,
       44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56.,
       57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69.,
       70., 71., 72., 73., 74., 75., 76., 77., 78., 79., 80., 81., 82.,
       83., 84., 85., 86., 87., 88., 89., 90., 91., 93., 97.])

In [22]:
movies.metascore.describe()

count    580.000000
mean      60.686207
std       16.557165
min        7.000000
25%       49.000000
50%       63.500000
75%       73.000000
max       97.000000
Name: metascore, dtype: float64

## `popularity`
Valor que vamos a descartar porque es muy volátil, cambia continuamente. Así, no guarda relación temporal con la taquilla que es el objeto de estudio.

In [23]:
# Convertirlo a número
movies['popularity'] = movies.popularity.str.replace(',', '').astype(float)

In [24]:
movies.popularity.describe()

count     311.000000
mean     2399.469453
std      1498.356966
min         8.000000
25%      1136.000000
50%      2317.000000
75%      3727.000000
max      4984.000000
Name: popularity, dtype: float64

In [25]:
movies[movies.title=='Five Feet Apart']

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world


## `directors`, `writers`, `stars`, `countries`, `companies`
Convertir a: comma separated values

In [26]:
movies['directors'] = movies['directors'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [27]:
movies['writers'] = movies['writers'].str.replace('[', '', regex=False) \
                                     .str.replace(']', '', regex=False) \
                                     .str.replace("'", "", regex=False) \
                                     .str.replace(", ", ",", regex=False)

In [28]:
movies['stars'] = movies['writers'].str.replace('[', '', regex=False) \
                                   .str.replace(']', '', regex=False) \
                                   .str.replace("'", "", regex=False) \
                                   .str.replace(", ", ",", regex=False)

In [29]:
movies['countries'] = movies['countries'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [30]:
movies['companies'] = movies['companies'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

## `opening_us_canada`, `gross_us_canada`, `gross_world`
Convertir estos valores a números, eliminando el signo del dólar

In [31]:
def remove_dolar(amount):
    if str(amount) == 'nan':
        return np.nan
    else:        
        return float(amount.replace(',', '')[1:]) 

In [32]:
movies['gross_us_canada'] = movies['gross_us_canada'].apply(remove_dolar)
movies['gross_us_canada']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
14627   NaN
14628   NaN
14629   NaN
14630   NaN
14631   NaN
Name: gross_us_canada, Length: 14494, dtype: float64

In [33]:
movies['opening_us_canada'] = movies['opening_us_canada'].apply(remove_dolar)
movies['opening_us_canada']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
14627   NaN
14628   NaN
14629   NaN
14630   NaN
14631   NaN
Name: opening_us_canada, Length: 14494, dtype: float64

In [34]:
movies['gross_world'] = movies['gross_world'].apply(remove_dolar)
movies['gross_world']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
14627   NaN
14628   NaN
14629   NaN
14630   NaN
14631   NaN
Name: gross_world, Length: 14494, dtype: float64

## `budget`
### Normalización `budget` a dólares
Son necesarios varios procesos para tener un valor válido de la variable `budget`:
- Separar, con regex, el texto que corresponde a la moneda y el que corresponde a la cantidad (ej. '€195,000'). Estos dos textos serán dos nuevas columnas `budgetCurrency` y `budgetAmount`
- Crear un diccionario con la correspondencia del código usado para las monedas en IMDb y el código ISO
- Crear un archivo externo que contenga las tasas de cambio para cada moneda y año
- Crear una nueva columna `budgetInDolars` con el valor ya normalizado a dólares

In [35]:
import re

def clean_amount_currency(raw_amount):
    
    if str(raw_amount) == 'nan' :
        return (np.nan, np.nan)
    
    # Diccionario con correspondencia del símbolo de moneda de IMDb y el ISO de monedas
    # Monedas de 2019 sin tasas de cambio: 'LVL'
    currency_to_iso = {'$':'USA', 'A$':'AUS', 'ZAR':'ZAF', '₹':'IND', '€':'EU27_2020',
                       'RUR':'RUS', 'R$':'BRA', 'CN¥':'CHN', 'TRL':'TUR', 'CA$':'CAN',
                       'NPR':'NPR', '£':'GBR', '¥':'JPN', 'HUF':'HUN', 'HRK':'HRV',
                       'NOK':'NOR', 'MNT':'MNT', 'CHF':'CHE', 'MX$':'MEX', 'UAH':'UAH',
                       'HK$':'HKG', 'PHP':'PHP', 'PLN':'POL', 'EGP':'EGP' , 'IRR':'IRR',
                       'NZ$':'NZL', 'AED':'AED', 'DKK':'DNK', 'NGN':'NGN', 'SEK':'SWE',
                       'BDT':'BDT', 'MYR':'MYR', 'CZK':'CZE', 'ARS':'ARG', 'PKR':'PKR',
                       'GEL':'GEL', 'NT$':'TWD', '₪':'ISR', '₫':'VND', '₩':'KOR', 'THB':'THB',
                       'RON':'ROU', 'PTE':'PRT', 'TTD':'TTD', 'MVR':'MVR', 'JOD':'JOD',
                       'ISK':'ISL', 'COP':'COL', 'LKR':'LKR', 'BSD':'BSD', 'BGL':'BGR',
                       'KZT':'KZT', 'KES':'KES', 'TZS':'TZS', 'DOP':'DOP', 'TJS':'TJS',
                       'TND':'TND', 'MUR':'MUR', 'SGD':'SGD', 'MOP':'MOP', 'IDR':'IDN',
                       'ETB':'ETB', 'CUP':'CUP', 'BYR':'BYR', 'MMK':'MMK', 'NAD':'NAD',
                       'PEN':'PER', 'MKD':'MKD', 'UGX':'UGX', 'BAM':'BAM', 'ALL':'ALL',
                       'KGS':'KGS', 'GHC':'GHS'
                      }
    
    raw_amount = raw_amount.replace(',', '')
    
    for item in re.finditer(r"^(\D+)(\d+)$", raw_amount):
        try:
            currency = currency_to_iso[item.group(1).strip()]
        except:
            currency = str(np.nan)
        amount = float(item.group(2))

    return(currency, amount)



# Dividir la info de budget en dos nuevas columnas (cantidad y moneda)
movies['budgetCurrency'], movies['budgetAmount'] = zip(*movies['budget'].map(clean_amount_currency))
movies[['title', 'budget', 'budgetAmount', 'budgetCurrency']].dropna()

# importar archivo csv de tasas de cambio
exchange_rates=pd.read_csv('../data/OECD/DP_LIVE_16072021155836489.csv')

# Seleccionar las tasas de cambio para el año del dataset
exchange_rates_anno = exchange_rates[exchange_rates.TIME == anno]

# Merge tasas de cambio con df movies, usando como primary key el código ISO de la moneda
movies = pd.merge(movies, exchange_rates_anno[['LOCATION','Value']], how='left', left_on='budgetCurrency', right_on='LOCATION')
movies['budgetInDollars'] = movies['budgetAmount'] / movies['Value']

movies.head()

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,...,awards,budget,gross_us_canada,opening_us_canada,gross_world,budgetCurrency,budgetAmount,LOCATION,Value,budgetInDollars
0,tt0062336,El Tango del Viudo y Su Espejo Deformante,NaN,2020.0,NaN,NaN,"Raoul Ruiz,Valeria Sarmiento","Raoul Ruiz,Omar Saavedra Santis","Raoul Ruiz,Omar Saavedra Santis",['Drama'],...,Awards\n1 nomination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0065392,Amintiri bucurestene,NaN,2020.0,NaN,NaN,"Yousif Mohamed Aidaby,Maria Alexandru,Radu Gabrea",NaN,NaN,['Documentary'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0093119,Grizzly II: El concierto,Grizzly II: The Predator,2020.0,Not Rated,1h 14min,André Szöts,"Joan McCall,David Sheldon","Joan McCall,David Sheldon","['Horror', 'Music', 'Thriller']",...,NaN,"$7,500,000",NaN,NaN,NaN,USA,7500000.0,USA,1.0,7500000.0
3,tt0293513,Pit Bull,NaN,2020.0,NaN,NaN,Aubrey K. Rattan,Aubrey K. Rattan,Aubrey K. Rattan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0368133,The Promise of Perfume,NaN,2020.0,NaN,NaN,Pavlina Proevska,Pavlina Proevska,Pavlina Proevska,['Documentary'],...,NaN,"$200,000",NaN,NaN,NaN,USA,200000.0,USA,1.0,200000.0


---
# Merge de tablas


In [36]:
imdb_merged = pd.merge(title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)] , title_ratings, on='tconst', how='left')
imdb_merged

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0.0,2020.0,NaN,70.0,Drama,6.3,140.0
1,tt0065392,movie,Bucharest Memories,Amintiri bucurestene,0.0,2020.0,NaN,NaN,Documentary,7.3,9.0
2,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0.0,2020.0,NaN,74.0,"Horror,Music,Thriller",3.3,658.0
3,tt0293513,movie,"Pit Bull: A Tale of Lust, Murder and Revenge",Pit Bull,0.0,2020.0,NaN,NaN,NaN,NaN,NaN
4,tt0368133,movie,The Promise of Perfume,The Promise of Perfume,0.0,2020.0,NaN,57.0,Documentary,6.6,5.0
...,...,...,...,...,...,...,...,...,...,...,...
14618,tt9913288,movie,Letters,Letters,0.0,2020.0,NaN,NaN,Drama,NaN,NaN
14619,tt9914192,movie,No Gogó do Paulinho,No Gogó do Paulinho,0.0,2020.0,NaN,98.0,Comedy,5.6,193.0
14620,tt9916190,movie,Safeguard,Safeguard,0.0,2020.0,NaN,90.0,"Action,Adventure,Thriller",3.6,139.0
14621,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0.0,2020.0,NaN,84.0,Thriller,5.8,1090.0


### Renombrar y seleccionar columnas de ambos datasets

In [37]:
imdb_merged.rename(columns={'primaryTitle': 'englishTitle',
                            'tconst': 'imdbId',
                            'averageRating': 'ratingImdb'
                          }, inplace=True)

In [38]:
imdb_merged.columns

Index(['imdbId', 'titleType', 'englishTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres', 'ratingImdb',
       'numVotes'],
      dtype='object')

In [39]:
movies.rename(columns={'imdb_id': 'imdbId',
                       'title': 'spanishTitle',
                       'budget': 'budgetString',
                       'budgetInDollars': 'budget',
                       'gross_us_canada': 'grossUsCanada',
                       'opening_us_canada': 'openingUsCanada',
                       'gross_world': 'grossWorld',
                      }, inplace=True)

In [40]:
movies.columns

Index(['imdbId', 'spanishTitle', 'original_title', 'year', 'certificate',
       'duration', 'directors', 'writers', 'stars', 'genres', 'countries',
       'companies', 'imdb_rating', 'metascore', 'popularity', 'awards',
       'budgetString', 'grossUsCanada', 'openingUsCanada', 'grossWorld',
       'budgetCurrency', 'budgetAmount', 'LOCATION', 'Value', 'budget'],
      dtype='object')

In [41]:
# Selección de columnas de imdb
imdb_merged = imdb_merged[['imdbId', 'englishTitle', 'originalTitle', 'isAdult', 'runtimeMinutes', 'genres', 'ratingImdb', 'numVotes']]

In [42]:
# Selección de columnas del escrapeado
movies = movies[['imdbId', 'spanishTitle', 'year', 'certificate',
                'directors', 'writers', 'stars', 'countries', 'companies',
                'metascore', 'awards', 'budget', 'grossUsCanada',
                'openingUsCanada', 'grossWorld']]

### Merge final para este año

In [43]:
movies_merged = pd.merge(imdb_merged, movies, on='imdbId', how='inner')
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14485 entries, 0 to 14484
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           14485 non-null  object 
 1   englishTitle     14485 non-null  object 
 2   originalTitle    14485 non-null  object 
 3   isAdult          14485 non-null  float64
 4   runtimeMinutes   10386 non-null  float64
 5   genres           14017 non-null  object 
 6   ratingImdb       6692 non-null   float64
 7   numVotes         6692 non-null   float64
 8   spanishTitle     14485 non-null  object 
 9   year             14485 non-null  float64
 10  certificate      1650 non-null   object 
 11  directors        14062 non-null  object 
 12  writers          12923 non-null  object 
 13  stars            12923 non-null  object 
 14  countries        14079 non-null  object 
 15  companies        10784 non-null  object 
 16  metascore        580 non-null    float64
 17  awards      

In [44]:
# Reordenar columnas

movies_merged = movies_merged[['imdbId', 'year', 'spanishTitle', 'originalTitle',
                               'englishTitle', 'ratingImdb', 'numVotes', 'metascore',
                               'isAdult', 'certificate', 'runtimeMinutes', 'genres', 'directors',
                               'writers', 'stars', 'countries', 'companies', 'awards',
                               'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld']]

In [45]:
movies_merged

,imdbId,year,spanishTitle,originalTitle,englishTitle,ratingImdb,numVotes,metascore,isAdult,certificate,...,directors,writers,stars,countries,companies,awards,budget,grossUsCanada,openingUsCanada,grossWorld
0,tt0062336,2020.0,El Tango del Viudo y Su Espejo Deformante,El Tango del Viudo y Su Espejo Deformante,The Tango of the Widower and Its Distorting Mi...,6.3,140.0,NaN,0.0,NaN,...,"Raoul Ruiz,Valeria Sarmiento","Raoul Ruiz,Omar Saavedra Santis","Raoul Ruiz,Omar Saavedra Santis",Chile,NaN,Awards\n1 nomination,NaN,NaN,NaN,NaN
1,tt0065392,2020.0,Amintiri bucurestene,Amintiri bucurestene,Bucharest Memories,7.3,9.0,NaN,0.0,NaN,...,"Yousif Mohamed Aidaby,Maria Alexandru,Radu Gabrea",NaN,NaN,Romania,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0093119,2020.0,Grizzly II: El concierto,Grizzly II: The Predator,Grizzly II: Revenge,3.3,658.0,7.0,0.0,Not Rated,...,André Szöts,"Joan McCall,David Sheldon","Joan McCall,David Sheldon","United States,Hungary",GBGB International,NaN,7.500000e+06,NaN,NaN,NaN
3,tt0293513,2020.0,Pit Bull,Pit Bull,"Pit Bull: A Tale of Lust, Murder and Revenge",NaN,NaN,NaN,0.0,NaN,...,Aubrey K. Rattan,Aubrey K. Rattan,Aubrey K. Rattan,United States,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0368133,2020.0,The Promise of Perfume,The Promise of Perfume,The Promise of Perfume,6.6,5.0,NaN,0.0,NaN,...,Pavlina Proevska,Pavlina Proevska,Pavlina Proevska,United States,Pavlina Ltd.,NaN,2.000000e+05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14480,tt9913288,2020.0,Letters,Letters,Letters,NaN,NaN,NaN,0.0,NaN,...,Dillon Jacobson,Dillon Jacobson,Dillon Jacobson,United States,NaN,NaN,NaN,NaN,NaN,NaN
14481,tt9914192,2020.0,No Gogó do Paulinho,No Gogó do Paulinho,No Gogó do Paulinho,5.6,193.0,NaN,0.0,NaN,...,Roberto Santucci,"Paulo Cursino,Odete Damico,Sergio Martorelli","Paulo Cursino,Odete Damico,Sergio Martorelli",Brazil,"Camisa Listrada,Panorama Filmes",NaN,NaN,NaN,NaN,NaN
14482,tt9916190,2020.0,Safeguard,Safeguard,Safeguard,3.6,139.0,NaN,0.0,NaN,...,Fraser Precious,Fraser Precious,Fraser Precious,United Kingdom,"GVN Releasing,High Octane Pictures",NaN,NaN,NaN,NaN,NaN
14483,tt9916270,2020.0,Il talento del calabrone,Il talento del calabrone,Il talento del calabrone,5.8,1090.0,NaN,0.0,NaN,...,Giacomo Cimini,"Giacomo Cimini,Lorenzo Collalti","Giacomo Cimini,Lorenzo Collalti","Italy,Spain","Paco Cinematografica,Eagle Pictures,Atica Cuar...",NaN,NaN,NaN,NaN,NaN


## Feature engineering. Columnas `profit` y `roi`
Los estadísticos están distorsionados por los valores nulos que se eliminarán después

In [46]:
movies_merged['profit'] = movies_merged.grossWorld - movies_merged.budget
movies_merged['profit'].describe()

count    2.120000e+02
mean     4.720750e+06
std      4.551008e+07
min     -1.332000e+08
25%     -3.510296e+06
50%     -9.214367e+05
75%     -2.505335e+04
max      3.814216e+08
Name: profit, dtype: float64

In [47]:
movies_merged['roi'] = (movies_merged.grossWorld - movies_merged.budget) / movies_merged.budget
movies_merged['roi'].describe()

count      212.000000
mean       436.721943
std       4656.030347
min         -0.999968
25%         -0.972425
50%         -0.850399
75%         -0.178088
max      59900.362137
Name: roi, dtype: float64

---
# Inspección inicial de correlaciones
## Registros no nulos con los que hacer las correlaciones de valoraciones y recaudación
Valoraciones: imdb_rating, metascore

Recaudación: budget, gross_world


In [48]:
print('Registros con ratingImdb:', len(movies_merged[movies_merged.ratingImdb.notnull()]))
print('Registros con metascore:', len(movies_merged[movies_merged.metascore.notnull()]))

print('Registros con budget:', len(movies_merged[movies_merged.budget.notnull()]))
print('Registros con grossUsCanada:', len(movies_merged[movies_merged.grossUsCanada.notnull()]))
print('Registros con openingUsCanada:', len(movies_merged[movies_merged.openingUsCanada.notnull()]))
print('Registros con grossWorld:', len(movies_merged[movies_merged.grossWorld.notnull()]))

print('\nRegistros con las rating, metascore, budget, grossWorld:',len(movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]))

Registros con ratingImdb: 6692
Registros con metascore: 580
Registros con budget: 4455
Registros con grossUsCanada: 236
Registros con openingUsCanada: 213
Registros con grossWorld: 1385

Registros con las rating, metascore, budget, grossWorld: 59


In [49]:
movies_merged = movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 11 to 14378
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           59 non-null     object 
 1   year             59 non-null     float64
 2   spanishTitle     59 non-null     object 
 3   originalTitle    59 non-null     object 
 4   englishTitle     59 non-null     object 
 5   ratingImdb       59 non-null     float64
 6   numVotes         59 non-null     float64
 7   metascore        59 non-null     float64
 8   isAdult          59 non-null     float64
 9   certificate      52 non-null     object 
 10  runtimeMinutes   59 non-null     float64
 11  genres           59 non-null     object 
 12  directors        59 non-null     object 
 13  writers          59 non-null     object 
 14  stars            59 non-null     object 
 15  countries        59 non-null     object 
 16  companies        59 non-null     object 
 17  awards        

## Correlaciones de valoraciones y recaudación

In [50]:
movies_merged.describe().round(2)

,year,ratingImdb,numVotes,metascore,isAdult,runtimeMinutes,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
count,59.0,59.00,59.00,59.00,59.0,59.00,5.900000e+01,3.900000e+01,39.00,5.900000e+01,5.900000e+01,59.00
mean,2020.0,5.83,35074.29,51.24,0.0,104.07,3.184621e+07,2.535233e+07,9081731.54,5.021655e+07,1.837034e+07,0.44
std,0.0,1.06,63373.17,17.26,0.0,19.42,4.844723e+07,4.287868e+07,14598086.16,1.051663e+08,8.174747e+07,2.99
min,2020.0,3.20,178.00,22.00,0.0,77.00,6.362338e+05,2.788000e+03,1052.00,9.270000e+02,-1.332000e+08,-1.00
25%,2020.0,5.05,1970.00,38.50,0.0,92.00,3.855954e+06,7.223815e+05,238930.50,3.630070e+05,-7.864890e+06,-0.92
50%,2020.0,6.00,10827.00,50.00,0.0,101.00,1.000000e+07,1.069109e+07,3600355.00,3.215636e+06,-2.526369e+06,-0.56
75%,2020.0,6.50,40695.00,66.50,0.0,109.00,3.900000e+07,2.169566e+07,9682181.00,4.323871e+07,6.560198e+06,0.37
max,2020.0,8.10,381262.00,87.00,0.0,201.00,2.050000e+08,2.063052e+08,62504105.00,4.614216e+08,3.814216e+08,19.45


In [51]:
method = 'pearson'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2020 Corr Método: pearson


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.714055,0.167408,0.258129,0.232864,0.140931
metascore,0.714055,1.000000,-0.000414,0.096683,0.124626,0.105739
budget,0.167408,-0.000414,1.000000,0.659904,0.256306,0.031187
grossWorld,0.258129,0.096683,0.659904,1.000000,0.895389,0.405115
profit,0.232864,0.124626,0.256306,0.895389,1.000000,0.502689
roi,0.140931,0.105739,0.031187,0.405115,0.502689,1.000000


In [52]:
method = 'kendall'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2020 Corr Método: kendall


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.531465,0.025550,0.088632,0.107540,0.096904
metascore,0.531465,1.000000,-0.099081,-0.015930,0.078470,0.011210
budget,0.025550,-0.099081,1.000000,0.611235,-0.175142,0.283284
grossWorld,0.088632,-0.015930,0.611235,1.000000,0.218001,0.673875
profit,0.107540,0.078470,-0.175142,0.218001,1.000000,0.450614
roi,0.096904,0.011210,0.283284,0.673875,0.450614,1.000000


In [53]:
method = 'spearman'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2020 Corr Método: spearman


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.699530,0.036657,0.104869,0.170025,0.125223
metascore,0.699530,1.000000,-0.194453,-0.056292,0.131037,0.032693
budget,0.036657,-0.194453,1.000000,0.801514,-0.129624,0.437432
grossWorld,0.104869,-0.056292,0.801514,1.000000,0.315546,0.863998
profit,0.170025,0.131037,-0.129624,0.315546,1.000000,0.634132
roi,0.125223,0.032693,0.437432,0.863998,0.634132,1.000000


In [54]:
movies_merged[['ratingImdb', 'metascore', 'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld', 'profit', 'roi']].corr()

,ratingImdb,metascore,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
ratingImdb,1.000000,0.714055,0.167408,0.284804,0.250714,0.258129,0.232864,0.140931
metascore,0.714055,1.000000,-0.000414,0.038635,0.009716,0.096683,0.124626,0.105739
budget,0.167408,-0.000414,1.000000,0.554067,0.460625,0.659904,0.256306,0.031187
grossUsCanada,0.284804,0.038635,0.554067,1.000000,0.970089,0.735544,0.675988,0.357349
openingUsCanada,0.250714,0.009716,0.460625,0.970089,1.000000,0.665040,0.632077,0.399767
grossWorld,0.258129,0.096683,0.659904,0.735544,0.665040,1.000000,0.895389,0.405115
profit,0.232864,0.124626,0.256306,0.675988,0.632077,0.895389,1.000000,0.502689
roi,0.140931,0.105739,0.031187,0.357349,0.399767,0.405115,0.502689,1.000000


---
# Guardado dataset a pickle

In [55]:
import pickle

with open('../data/web_imdb_clean/movies_2020.pickle', 'wb') as f:
    pickle.dump(movies_merged, f)